In [1]:
import os
import time
import json
import tqdm
import geocoder
import pandas as pd
from pprint import pprint

In [2]:
BAIDU_AK = '62c28637e66010289bdef04310267902'
COUNTRY_JSON_FILENAME = 'country.json'
PROVINCES_JSON_FILENAME = "provinces.json"

In [3]:
last_ncov_json = sorted([e for e in os.listdir('.') if e.startswith('ncov_')], reverse=True)[0]
print(last_ncov_json)
country_df = pd.read_json(last_ncov_json)
country_df

ncov_20200206-093911.json


,provinceName,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,cities
0,湖北省,湖北,19665,0,633,549,,420000,"[{'cityName': '武汉', 'confirmedCount': 10117, '..."
1,广东省,广东,944,0,49,0,,440000,"[{'cityName': '深圳', 'confirmedCount': 314, 'su..."
2,浙江省,浙江,895,0,78,0,,330000,"[{'cityName': '温州', 'confirmedCount': 364, 'su..."
3,河南省,河南,851,0,54,2,,410000,"[{'cityName': '信阳', 'confirmedCount': 164, 'su..."
4,湖南省,湖南,711,0,56,0,,430000,"[{'cityName': '长沙', 'confirmedCount': 179, 'su..."
5,安徽省,安徽,591,0,23,0,,340000,"[{'cityName': '合肥', 'confirmedCount': 104, 'su..."
6,江西省,江西,548,0,27,0,,360000,"[{'cityName': '南昌', 'confirmedCount': 134, 'su..."
7,重庆市,重庆,389,0,15,2,,500000,"[{'cityName': '万州区', 'confirmedCount': 68, 'su..."
8,江苏省,江苏,373,0,26,0,,320000,"[{'cityName': '苏州', 'confirmedCount': 60, 'sus..."
9,山东省,山东,343,0,18,0,,370000,"[{'cityName': '青岛', 'confirmedCount': 41, 'sus..."


In [4]:
provinces_df = pd.DataFrame()
for idx in country_df.index:
    se = country_df.iloc[idx]
    if se['cities']:
        provinceName = se['provinceName']
        cities_df = pd.read_json(json.dumps(se['cities']))
        with tqdm.tqdm(cities_df.index) as pbar:
            for idx in pbar:
                se = cities_df.iloc[idx].copy()
                pbar.set_description('{} {}'.format(provinceName, se['cityName']))
                g = geocoder.baidu(u"{} {}".format(provinceName, se['cityName']), key=BAIDU_AK)
                se['provinceName'] = provinceName
                se['lat'] = g.latlng[0]
                se['lng'] = g.latlng[1]
                provinces_df = provinces_df.append(se, ignore_index=True)

西藏自治区 拉萨: 100%|███████████████████████████| 1/1 [00:00<00:00, 14.43it/s]       


In [5]:
provinces_df

,cityName,confirmedCount,curedCount,deadCount,lat,lng,locationId,provinceName,suspectedCount
0,武汉,10117.0,432.0,414.0,30.598467,114.311582,420100.0,湖北省,0.0
1,孝感,1886.0,9.0,25.0,30.930689,113.922510,420900.0,湖北省,0.0
2,黄冈,1807.0,62.0,29.0,30.459359,114.878490,421100.0,湖北省,0.0
3,随州,834.0,9.0,9.0,31.696517,113.389450,421300.0,湖北省,0.0
4,荆州,801.0,18.0,10.0,30.340842,112.245523,421000.0,湖北省,0.0
...,...,...,...,...,...,...,...,...,...
405,第六师,1.0,0.0,0.0,43.799238,87.633473,0.0,新疆维吾尔自治区,0.0
406,巴州,1.0,0.0,0.0,31.857686,106.775513,511902.0,新疆维吾尔自治区,0.0
407,西宁,14.0,3.0,0.0,36.623385,101.784450,630100.0,青海省,0.0
408,海北州,3.0,0.0,0.0,36.960663,100.907434,632200.0,青海省,0.0


In [6]:
country_df.to_json(open(COUNTRY_JSON_FILENAME, 'w'))
provinces_df.to_json(open(PROVINCES_JSON_FILENAME, 'w'))